In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

/Users/sgkang/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/sgkang/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: tcp://127.0.0.1:61055 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 8.59 GB


In [2]:
import time
import random

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

In [3]:
%time costly_simulation([1, 2, 3, 4])

CPU times: user 22 ms, sys: 3.78 ms, total: 25.8 ms
Wall time: 617 ms


10

In [4]:
import pandas as pd
import numpy as np

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

,param_a,param_b,param_c,param_d
0,0.813553,0.072500,0.505994,0.591008
1,0.597512,0.870158,0.413977,0.780401
2,0.422417,0.549025,0.142763,0.815552
3,0.479216,0.127857,0.092610,0.692925
4,0.622246,0.340204,0.097761,0.995724


In [5]:
results = []

In [6]:
%%time
for parameters in input_params.values[:10]:
    result = costly_simulation(parameters)
    results.append(result)

CPU times: user 181 ms, sys: 22.3 ms, total: 203 ms
Wall time: 6.44 s


In [7]:
results

[1.9830557150573547,
 2.6620487427372126,
 1.9297570164417226,
 1.3926081438954812,
 2.055935412405419,
 1.4889200468332824,
 2.630269818540213,
 2.8523951532147493,
 2.910427867930593,
 2.2657621780179094]

In [8]:
import dask
lazy_results = []

In [9]:
%%time

for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

CPU times: user 2.38 ms, sys: 2.51 ms, total: 4.89 ms
Wall time: 3.65 ms


In [10]:
%time dask.compute(*lazy_results)

CPU times: user 121 ms, sys: 15.7 ms, total: 136 ms
Wall time: 1.36 s


(1.9830557150573547,
 2.6620487427372126,
 1.9297570164417226,
 1.3926081438954812,
 2.055935412405419,
 1.4889200468332824,
 2.630269818540213,
 2.8523951532147493,
 2.910427867930593,
 2.2657621780179094)